<a href="https://colab.research.google.com/github/kaispace30098/Project-Viking-Puffin/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Training a Custom Character LoRA with DreamBooth and SDXL**
This notebook walks through the end-to-end process of fine-tuning the Stable Diffusion XL 1.0 model to recognize a new, custom character. We use a combination of DreamBooth and Low-Rank Adaptation (LoRA) for parameter-efficient fine-tuning.

In [ ]:
# 1. Mount Google Drive to access our images and save the output
from google.colab import drive
drive.mount('/content/drive')

###Step 1: Prepare the Training Data
Before running this notebook, the training data must be prepared.

1.Collect Images: Gather 15-20 photos of your subject.

2.Preprocess: Crop and resize these images to a square aspect ratio. While the model trains at 1024x1024, preprocessing them to a consistent size like 1024x1024 or 512x512 is recommended. I used the free online tool BIRME for this.

3.Upload: Place the processed images in a folder on your Google Drive. The path to this folder will be used in the training command.

In [ ]:
# (Optional) Run this code to verify the size of your preprocessed images
# import os
# from PIL import Image
#
# img_dir = "/content/drive/MyDrive/MyLoRA/character_images"
# for filename in os.listdir(img_dir):
#     if filename.lower().endswith((".jpg", ".png", "jpeg")):
#         path = os.path.join(img_dir, filename)
#         with Image.open(path) as img:
#             print(f"{filename}: {img.size}")

##Step 2: Environment Setup
Here, we install all the necessary Python libraries.

Note: I am installing diffusers directly from its GitHub source. This is crucial because the training script from the repository's main branch often requires features not yet available in the stable pip release, preventing version mismatch errors.

In [ ]:
# Install required libraries
print("Installing libraries...")
!pip install -q transformers accelerate bitsandbytes peft safetensors xformers
# Install diffusers from the main branch on GitHub
!pip install -q git+https://github.com/huggingface/diffusers.git
print("Libraries installed successfully.")

###Step 3: Configure Environment & Clone Repository
First, we configure accelerate, which is a library from Hugging Face that simplifies running PyTorch scripts on any infrastructure (like Colab's single GPU).

Second, we clone the diffusers repository to get access to the train_dreambooth_lora_sdxl.py training script.

In [ ]:
# Configure Accelerate for Colab's single-GPU environment
!accelerate config default
print("Accelerate configured.")

# Clone the diffusers repository to get the training script
import os
if not os.path.exists('/content/diffusers'):
  print("Cloning diffusers repository...")
  !git clone https://github.com/huggingface/diffusers.git
else:
  print("Diffusers repository already exists.")

###Step 4: Define Training Parameters
We use environment variables to keep the main training command clean and easy to read.

MODEL_NAME: The base model we are fine-tuning (SDXL 1.0).

INSTANCE_DIR: The path to our folder of training images on Google Drive.

OUTPUT_DIR: The path where the trained LoRA and checkpoints will be saved.

INSTANCE_PROMPT: The unique prompt that will trigger our character. This prompt creates the association between a new "word" (Kaiffin) and our images.

In [ ]:
# Define environment variables
%env MODEL_NAME=stabilityai/stable-diffusion-xl-base-1.0
%env INSTANCE_DIR=/content/drive/MyDrive/MyLoRA/character_images
%env OUTPUT_DIR=/content/drive/MyDrive/MyLoRA/output_lora
%env INSTANCE_PROMPT=a photo of Kaiffin

###Step 5: Launch Training

This is the final step. We use accelerate launch to execute the training script with all our specified parameters. The training will run for 1000 steps and save a checkpoint every 500 steps.

In [ ]:
# Final corrected training command
!accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=1024 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --checkpointing_steps=500 \
  --validation_epochs=50 \
  --validation_prompt="photo_of_Kaiffin_on_an_icy_cliff" \
  --seed="0" \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_checkpointing \
  --enable_xformers_memory_efficient_attention \
  --report_to="tensorboard"